In [11]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spam-filter/emails.csv


In [12]:
import numpy as np
import pandas as pd
import nltk
import string
from nltk.corpus import stopwords

# detecting if the email is spam or not

In [13]:
df=pd.read_csv('/kaggle/input/spam-filter/emails.csv')
df

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1
...,...,...
5723,Subject: re : research and development charges...,0
5724,"Subject: re : receipts from visit jim , than...",0
5725,Subject: re : enron case study update wow ! a...,0
5726,"Subject: re : interest david , please , call...",0


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5728 entries, 0 to 5727
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    5728 non-null   object
 1   spam    5728 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 89.6+ KB


In [15]:
df.duplicated().sum()

33

In [16]:
df.drop_duplicates(inplace=True)

In [17]:
df.shape

(5695, 2)

In [18]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# remove stopwords and punctuation

In [19]:
def process_text(text):
    # remove punctuation
    nopunc=[char for char in text if char not in string.punctuation]
    nopunc=''.join(nopunc)
    # remove stopwords
    clean_words=[word for word in nopunc.split() if word.lower()not in stopwords.words('english')]
    return clean_words

#  text tokenization

In [20]:
df['text'].head().apply(process_text)

0    [Subject, naturally, irresistible, corporate, ...
1    [Subject, stock, trading, gunslinger, fanny, m...
2    [Subject, unbelievable, new, homes, made, easy...
3    [Subject, 4, color, printing, special, request...
4    [Subject, money, get, software, cds, software,...
Name: text, dtype: object

# convert a text to matrix of tokens

In [22]:
from sklearn.feature_extraction.text import CountVectorizer
message_bow=CountVectorizer(analyzer=process_text).fit_transform(df['text'])

# split the data

In [23]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(message_bow,df['spam'],test_size=.25,random_state=42)

In [24]:
message_bow.shape

(5695, 37229)

# Naive Bayes Classifier

In [25]:
from sklearn.naive_bayes import MultinomialNB
classifier=MultinomialNB().fit(x_train,y_train)
classifier.score(x_train,y_train)

0.9969562163427769

# evaluating the training accuracy

In [28]:
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix
pred=classifier.predict(x_train)
print(classification_report(pred,y_train))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3265
           1       1.00      0.99      0.99      1006

    accuracy                           1.00      4271
   macro avg       1.00      0.99      1.00      4271
weighted avg       1.00      1.00      1.00      4271



# the models accuracy

In [30]:
y_pred=classifier.predict(x_test)
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      1045
           1       0.99      0.97      0.98       379

    accuracy                           0.99      1424
   macro avg       0.99      0.99      0.99      1424
weighted avg       0.99      0.99      0.99      1424



# comparing between prediction and real data

In [36]:
print(y_pred[:10])
print()
print(y_test.values[:10])

[0 1 0 0 0 1 0 0 0 0]

[0 1 0 0 0 1 0 0 0 0]
